# Sorting

In this lecture we will focus on algorithms to sort elements. The sorting algorithms we will cover in order of descending time complexity

- Insertion Sort
- Bubble Sort
- Shellsort
- Heapsort
- Merge Sort 
- Quicksort
- Bucket Sort
- Radix Sort

We will also cover external sorting for when an object is too large to fit into memory. 

## Insertion Sort
One of the simpliest sorting algorithms is **insertion sort**. Insertion sort consist of N-1 **passes**. At each pass, p, we ensure that the array from 0 to p is in sorted order. Since we know that all the elements from 0 to p-1 are in sorted order we need to find out where element p belongs in the already sorted list. So in each pass p we move the element in position p left until it's in the correct position. Lets look at how this is implemented
```python
def insertion_sort(arr): 
    # iterate through the array
    for i in range(1, len(arr)): 
  
        key = arr[i] 
  
        # Move elements of arr[0..i-1], that are 
        # greater than key, to one position ahead 
        # of their current position 
        j = i
        while j > 0 and key < arr[j-1] : 
                arr[j] = arr[j-1] 
                j -= 1
        arr[j] = key
    return arr
```
Because of the nested loops, each of which can take N iterations, insertion sort runs in $O(N^2)$. We could easily see this is possible if the input is reversed. However if the list is already sorted then the inner while loop never runs hence giving it O(N) running time. 
Lets look at a visualization of the algorithm. 
![SegmentLocal](./files/Sorting/insertion_sort.gif "segment")

## Bubble Sort
Another extremely simple sorting algorithm is **bubble sort**. Bubble sort works by repeatedly stepping through the list and comparing each pair of adjacent elements and swapping them if they are in the wrong order. The algorithm continually steps through the list until no swaps are performed during a pass. During each pass bubble sort is finding the $n^{th}$ largest element and putting it into the correct place. Similar to insertion sort, the running time of this algorithm is $O(N^2)$. Hence if the list is in reversed order it will take N passes to find the smallest element in the list and put it in the correct position. Let's look at how bubble sort is implemented
```python
def bubble_sort(arr):
    # There is also no do while in python
    # so we will emulate one by initializing swapped to True
    # and setting it to false at the beginning of the loop
    swapped = True
    while swapped:
        swapped = False
        # Perform a pass on the array 
        for i in range(len(arr)-1):
            # If the elements are in the wrong order swap them
            # set swapped to True as a swap has been performed
            if arr[i] > arr[i+1]:
                arr[i], arr[i+1] = arr[i+1], arr[i]
                swapped = True
    return arr
```
Lets look at a visualization of the algorithm in work.
![SegmentLocal](./files/Sorting/bubble_sort.gif "segment")

## Shell Sort
Shell sort takes the idea of insertion sort and generalizes it. The idea is that to arrange the list of elements so that starting from anywhere every $h^{th}$ element from that position is sorted, or we can state that for every i, a\[i\] $\le$ a\[i+h\]. The list at this point is said to be **h-sorted**. The algorithm starts by initially considering a large h and then decreases until the last phase in which h=1. The question is how to determine the sequence of h (gaps) to use when performing the sort. Let's look at an example if we chose 5,3,1 as the gaps.

<img src="./files/Sorting/shellsort.png" width="600"/>


Shell suggest $h_1 = N/2$, and $h_k = N/2^k$. However the problem with Shell's method is that is has a running time of $O(N^2)$ so there is no obvious case for using it over insertion or bubble sort. We can show that the upper bound using Shell's method is $O(N^2)$. Each pass with an increment of $h_k$ consists of $h_k$ insertion sorts of $N/h_k$ elements. Since insertion sort is quadratic, the total cost of a pass is $O(h_k(N/h_k)^2) = O(N^2/h_k)$. Summing over all the passes gives a total bound of $\sum_{i=1}^{t} N^2/h_i = N^2\sum_{i=1}^{t} 1/h_i$. Since the increments form a geometric series with a common ratio of 2, and the largest term in the series is 1, $\sum_{i=1}^{t} 1/h_i < 2$. This we obtain a upper bound of $O(N^2)$.

The problem with Shell's method is that the increments are not relatively prime, and thus smaller increments can have little effect. Hibbard sugggest a slightly different increment sequence, giving a better run time of $O(N^{3/2})$. Hibbard's method states to use increments of the form $1,3,7,...,2^k-1$. The key difference here is that each increment is prime therefore they have no common factors between the increments. The proof of this requires using additive number theory which is beyond the scope of this class and therefore left out. 

There are other methods that have been applied to shell sort not just Shell's and Hibbard's. Prat showed that using $O(N^{3/2})$ applies to a wide range of increment sequences. Whereas Sedgewick has proposed multiple increment sequences that give an $O(N^{4/3})$ worst case running time using $4^k+3 * 2^i + 1$. There are several more that use theorems from number theory and combinatorics such as Cuira's method which has yet to have a proven run time. Hence why the actual running time of shell sort is unknown.

Let's look at code for using Hibbards algorithm
```python
def shell_sort(arr):
    # Create the increment sequence
    gaps = []
    for i in range(1, int(log(len(arr),2))):
        gaps.insert(0,(2 ** i - 1))
    
    for gap in gaps:
        # perform generalized insertion sort for that gap
        for i in range(gap, len(arr)):
            key = arr[i]
            j = i
            while j >= gap and key < arr[j-gap]:
                arr[j] = arr[j-gap]
                j -= gap
            
            arr[j] = key
        
    return arr
```

## Heap Sort
Binary Heaps (Priority Queues) can be used to sort in O(NlogN) time. So far this algorithm gives the best running time for sorting algorithms. Recall that the basic strategy of a min heap is that the minimum element is always at the root. So the idea is that given an unsorted list to build a binary heap of N elements (this takes O(N) time). We then perform N deleteMin operations getting the elements in sorted order. Each time we perform a deleteMin we save the result in a second array to hold the sorted elements. Since the running time of deleteMin is O(logN) and we perform it N times the total running time is O(NlogN). <br>
The only problem with this algorithm is space complexity since it uses a second array to hold the sorted items. This is not usually a problem as memory is cheap and plentiful but could become an issue if the input is substantially large. A clever way to avoid using a second array is to use the array representing the binary heap to maintain the sorted order. Since after each delete the heap decreases size by 1 the cell that was last in the heap could be used to store the removal. If we were to use this strategy on a min heap it would return an array of items sorted in reverse order. To fix this instead of using a min heap we now use a max heap. Assuming we are using a max heap here is how we would implement a heap sort inside the heap class. We will use the same stucture as the binary heap class we implemented in the heaps lecture.
```python
def heap_sort(arr):
    # first build a heap
    self.build_heap(arr)
    
    # perform N delete maxes and place the max element in the spot that was just removed
    # This sorts the array that the heap is built on
    for i in range(len(self.heap)):
        element = self.delete_max()
        self.heap[self.size+1] = element
    
    # The array maintaining the heap is now sorted
    # We remove the first element since the indexes in a heap start at 1
    return self.heap[1:]
```

## Merge Sort 